# EXPERIMENTS

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

In [4]:
from torchtext.datasets import SQuAD1

In [86]:
train, test = SQuAD1("root")

In [6]:
from modules.data import *

#get_dataframe,  get_pairs_from_df, cols, sample_df_perc, get_thresholds, get_outliers, tokenize_sentence, remove_least_common, to_tensor,  filter_sentences

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nebelgrau/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nebelgrau/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nebelgrau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# get train and test dataframes of sentences
train_df, test_df = get_dataframe(train), get_dataframe(test)

In [9]:
train_df.shape, test_df.shape

((87599, 2), (10570, 2))

In [10]:
train_len = 15000
test_len = 3000

In [11]:
train_df = sample_df_num(train_df, train_len)

In [12]:
test_df = sample_df_num(test_df, test_len)

In [13]:
train_df.shape, test_df.shape

((15000, 2), (3000, 2))

In [14]:
train_df.head(3)

,Question,Answer
0,In approximately what year did a movement in f...,1890
1,When were skills that guaranteed survival in t...,the medieval period
2,The Statue of Liberty is also in what other US...,New Jersey


In [15]:
from torchtext.data.utils import get_tokenizer

In [16]:
tokenizer = get_tokenizer("basic_english")

In [18]:
tokenizer(train_df['Question'][77])

['what',
 'was',
 'the',
 'name',
 'of',
 'the',
 'new',
 'musical',
 'style',
 'that',
 'emerged',
 'from',
 'new',
 'york',
 'in',
 'the',
 '1970s',
 '?']

In [19]:
from torchtext.vocab import build_vocab_from_iterator

In [44]:
count = 0
for _, q, a, _ in test:    
    print(q,a)
    count += 1
    if count > 3:
        break

Which NFL team represented the AFC at Super Bowl 50? ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
Which NFL team represented the NFC at Super Bowl 50? ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']
Where did Super Bowl 50 take place? ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]
Which NFL team won Super Bowl 50? ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']


In [54]:
def yield_tokens(data_iter):
    for _, question, answer, _ in data_iter:
        # yield (tokenizer(question), tokenizer(answer[0]))
        yield tokenizer(question)        
    

In [82]:
testdata = yield_tokens(test)

In [56]:
def get_vocab(train_datapipe):
    vocab = build_vocab_from_iterator(yield_tokens(train_datapipe),
                                      specials=['<SOS>', '<EOS>', '<PAD>', '<UNK>'])
    vocab.set_default_index(vocab['<UNK>'])
                                                
    return vocab

In [58]:
Q_vocab = get_vocab(test)

In [66]:
Q_vocab['man']

844

In [67]:
from torch.nn.utils.rnn import pad_sequence

In [87]:
count = 0
testseq = []
for n in yield_tokens(test):
    print(n)
    count += 1
    testseq = n
    if count > 0:
        
        break

['which', 'nfl', 'team', 'represented', 'the', 'afc', 'at', 'super', 'bowl', '50', '?']


In [88]:
testseq

['which',
 'nfl',
 'team',
 'represented',
 'the',
 'afc',
 'at',
 'super',
 'bowl',
 '50',
 '?']

In [89]:
pad_sequence(testseq)

TypeError: expected Tensor as element 0 in argument 0, but got str

In [90]:
pad_sequence?

Signature:
pad_sequence(
    sequences: Union[torch.Tensor, List[torch.Tensor]],
    batch_first: bool = False,
    padding_value: float = 0.0,
) -> torch.Tensor
Docstring:
Pad a list of variable length Tensors with ``padding_value``

``pad_sequence`` stacks a list of Tensors along a new dimension,
and pads them to equal length. For example, if the input is list of
sequences with size ``L x *`` and if batch_first is False, and ``T x B x *``
otherwise.

`B` is batch size. It is equal to the number of elements in ``sequences``.
`T` is length of the longest sequence.
`L` is length of the sequence.
`*` is any number of trailing dimensions, including none.

Example:
    >>> from torch.nn.utils.rnn import pad_sequence
    >>> a = torch.ones(25, 300)
    >>> b = torch.ones(22, 300)
    >>> c = torch.ones(15, 300)
    >>> pad_sequence([a, b, c]).size()
    torch.Size([25, 3, 300])

Note:
    This function returns a Tensor of size ``T x B x *`` or ``B x T x *``
    where `T` is the length of th